##### doc - https://developers.transistor.fm/#episodes

In [ ]:
!pip install pandas-gbq
!pip install google-cloud-secret-manager
!pip install google-cloud-storage
!pip install google-api-core==2.8.0

In [ ]:
# usando a lib storage do GCP (bucket)
from google.cloud import storage

# Instanciando client do storage
client = storage.Client(project="dev-stalse")

for atual_file in client.list_blobs('dev-stalse-us-notebooks', prefix='dependencies_podlink'):
    _a = str(atual_file).split(",")
    _folder_atual = "gs://dev-stalse-us-notebooks/"
    
    if "stalse_functions.py" in _a[1]:
        path_atual = _folder_atual+_a[1]
        path_atual = path_atual.replace(" ", "")
        path_atual = path_atual.split("/")
        nome_script = path_atual[4]
        
        with open(nome_script, 'wb') as file_obj:
            client.download_blob_to_file(
                'gs://dev-stalse-us-notebooks/dependencies_podlink/{}'.format(nome_script), file_obj)

In [ ]:
# import libs
import requests, json
import stalse_functions as sf
import pandas as pd

In [ ]:
# obtendo episódios
def req_apisodes(page=0):
    """Função que recupera uma lista paginada de episódios em ordem decrescente por data publicada

    Args:
        per (int, optional): Quantidade de resultados por página. Defaults to 10.
        page (int, optional): Número da página. Defaults to 0.

    Returns:
        json: Informações descritivas de cada episódio
    """
    
    # utilizando a api-key do KMS
    apikey = sf.keygcp('msc-apikey-transistor')
    
    # tratando paginação
    if page==0:
        url = "https://api.transistor.fm/v1/episodes?pagination[per]=10&pagination[page]=1"
    
    else:
        url = "https://api.transistor.fm/v1/episodes?pagination[per]=10&pagination[page]="+str(page)

    payload={}
    headers = {
    'x-api-key': apikey
    }

    response = requests.get(url, headers=headers, data=payload)
    
    # transformando a resposta da requisição em json
    res = response.text
    res = json.loads(res)
    return res

In [ ]:
# totalizando episódios para usar no 'for' futuramente e com este número tratar a paginação
total_valores = req_apisodes()['meta']['totalCount']

In [ ]:
# buscando apenas as colunas necessárias
df_cols = ['id']
df_inner_cols = ['title', 'number', 'season', 'status', 'published_at', 'duration', 'explicit', 'keywords', 'alternate_url', 'media_url', 
                 'image_url', 'author', 'summary', 'created_at', 'updated_at', 'duration_in_mmss', 
                 'share_url', 'audio_processing', 'type', 'email_notifications']

In [ ]:
# percorrendo os níveis do json para gerar dados e popular no df futuramete
epsodes_data = []

for i in range(1, total_valores):
    atual_value=req_apisodes(i)
    
    # nível 1 do json
    for epsode in atual_value['data']:
        _c = []
        for col in df_cols:
            try:
                _c.append(epsode[col])
            except KeyError:
                _c.append(None)
        
        # nível 2 do json            
        for innercol in df_inner_cols:
            try:
                _c.append(epsode['attributes'][innercol])
            except KeyError:
                _c.append(None)
        
        # informação aglutinada dos dois níveis do json
        epsodes_data.append(_c)

In [ ]:
# jutando as colunas de diferentes níveis do json 
ndf = df_cols + df_inner_cols

# criando df com as informações obtidas acima
df_episodes = pd.DataFrame(data=epsodes_data, columns=ndf)

In [ ]:
# obtendo id's do BigQuery
id_tabela = "`mosaic-fertilizantes.podcast.episodes`"
project = id_tabela.split('.')[0]

In [ ]:
# deletando SOMENTE os ID's que a API retornou
sf.deleta_ids(df_episodes, 'id', id_tabela, project)

In [ ]:
# criando e consumindo csv para importação no BigQuery
csv = sf.csv_bucket(df_episodes, 'episodes')

In [ ]:
table_schema=[
{'name': 'id','type': 'INTEGER'},
{'name': 'title','type': 'STRING'},
{'name': 'number','type': 'INTEGER'},
{'name': 'season','type': 'INTEGER'},
{'name': 'status','type': 'STRING'},
{'name': 'published_at','type': 'STRING'},
{'name': 'duration','type': 'INTEGER'},
{'name': 'explicit','type': 'BOOLEAN'},
{'name': 'keywords','type': 'STRING'}, # API mudou de este campo de float para string (18/07)
{'name': 'alternate_url','type': 'FLOAT'},
{'name': 'media_url','type': 'STRING'},
{'name': 'image_url','type': 'STRING'},
{'name': 'author','type': 'STRING'}, # API mudou de este campo de float para string (15/08)
{'name': 'summary','type': 'STRING'}, # API mudou de este campo de float para string (29/08)
{'name': 'created_at','type': 'STRING'},
{'name': 'updated_at','type': 'STRING'},
{'name': 'duration_in_mmss','type': 'STRING'},
{'name': 'share_url','type': 'STRING'},
{'name': 'audio_processing','type': 'BOOLEAN'},
{'name': 'type','type': 'STRING'},
{'name': 'email_notifications','type': 'FLOAT'}
]

In [ ]:
# criando a tabela no BigQuery
sf.cria_bq(df_episodes, id_tabela, 'append', project, csv, "us-east4", table_schema)